In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.current_device()

0

In [4]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce MX230'

In [5]:
torch.cuda.memory_allocated()

0

In [6]:
torch.cuda.memory_reserved()

0

In [7]:
a = torch.FloatTensor([1.0,2.0])

In [8]:
a.device

device(type='cpu')

In [9]:
a = torch.FloatTensor([1.0,2.0]).cuda()

In [10]:
a.device

device(type='cuda', index=0)

In [11]:
torch.cuda.memory_allocated()

512

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\Seedevice\anaconda3\envs\pytorchenv\lib\site-packages\sklearn\utils\validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


In [13]:
class Model(nn.Module):
    
    def __init__(self, in_features = 4, h1 = 8, h2 = 9, out_features = 3):
        # how many Layers?
        # Input Layer(4 features) --> h1 N --> h2 N --> output(3 classes)
       
        super().__init__()
        self.fc1 = nn.Linear(in_features, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.out = nn.Linear(h2, out_features)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

In [14]:
torch.manual_seed(32)
model = Model()

In [15]:
next(model.parameters()).is_cuda

False

In [16]:
gpumodel = model.cuda()

In [17]:
next(gpumodel.parameters()).is_cuda

True

In [18]:
df = pd.read_csv('../Pytorch Tutorial/ANN-Artificial-Neural-Networks/iris.csv')

In [19]:
X = df.drop('target', axis = 1).values

In [20]:
X

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [21]:
y = df['target'].values

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=33)

In [23]:
X_train = torch.FloatTensor(X_train).cuda()

In [24]:
X_test = torch.FloatTensor(X_test).cuda()
y_train = torch.LongTensor(y_train).cuda()
y_test = torch.LongTensor(y_test).cuda()

In [25]:
trainloader = DataLoader(X_train, batch_size = 60, shuffle = True, pin_memory = True)
testloader = DataLoader(X_test, batch_size = 60, shuffle = True, pin_memory = True)

In [26]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr= 0.01)

In [28]:
import time

start = time.time()
epochs = 100
losses = []

for i in range(epochs):
    
    
    
    y_pred = gpumodel.forward(X_train)
    loss = criterion(y_pred,y_train)
    losses.append(loss)
    
    if i%10 == 0:
        print(f'Epoch: {i} loss {loss.item()}')
        
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    

end = time.time() - start
print(f"Total time: {end}")

Epoch: 0 loss 1.1507112979888916
Epoch: 10 loss 0.9377313256263733
Epoch: 20 loss 0.7798258662223816
Epoch: 30 loss 0.6099399924278259
Epoch: 40 loss 0.40079933404922485
Epoch: 50 loss 0.2543632388114929
Epoch: 60 loss 0.1505305916070938
Epoch: 70 loss 0.10086949169635773
Epoch: 80 loss 0.08128315955400467
Epoch: 90 loss 0.0723142996430397
Total time: 0.33449554443359375


In [30]:
correct = 0
with torch.no_grad():
    for i, data in enumerate(X_test):
        y_val = gpumodel.forward(data)
        print(f"{i+1:2}. {str(y_val):38} {y_test[i]}")
        if y_val.argmax().item() == y_test[i]:
            correct +=1
print(f'\n{correct} out of {len(y_test)} = {100*correct/len(y_test):.2f}% correct' )

 1. tensor([-2.1252,  4.8064, -0.8628], device='cuda:0') 1
 2. tensor([-1.7985,  5.3098, -1.5449], device='cuda:0') 1
 3. tensor([  6.3542,   0.8438, -10.0541], device='cuda:0') 0
 4. tensor([-3.9123,  4.5958,  1.1693], device='cuda:0') 1
 5. tensor([-7.4713,  3.2021,  5.7853], device='cuda:0') 2
 6. tensor([-10.4976,   1.6459,   9.6297], device='cuda:0') 2
 7. tensor([  6.3201,   0.9917, -10.1532], device='cuda:0') 0
 8. tensor([  7.0468,   0.7059, -10.9137], device='cuda:0') 0
 9. tensor([-7.2061,  3.3477,  5.3565], device='cuda:0') 2
10. tensor([-9.3960,  2.5759,  8.1033], device='cuda:0') 2
11. tensor([-9.8808,  2.3475,  8.7141], device='cuda:0') 2
12. tensor([ 6.2748,  0.6655, -9.7613], device='cuda:0') 0
13. tensor([-9.3142,  2.1880,  8.1947], device='cuda:0') 2
14. tensor([-3.7803,  4.5050,  1.0752], device='cuda:0') 1
15. tensor([-7.8657,  3.0117,  6.2303], device='cuda:0') 2
16. tensor([-1.8867,  5.1572, -1.3345], device='cuda:0') 1
17. tensor([-5.7006,  3.5030,  3.6696], devi